# Determining the origin of the broad component in "normal" star-forming galaxies using a matched-sample analysis
---
From Kaasisnen et al. (2017):
> *We select three ‘matched’ samples from the full local sample by matching **an ensemble of local counterparts to each galaxy in our z ∼ 1.5 [O II]-Hα sample**. Our first local comparison sample is matched solely on stellar mass, without applying any constraints to the SFR. We refer to this sample as the $M_∗$-matched sample...  To create the $M_∗$-matched sample, **we require the stellar mass of the local counterparts to be within 0.2 dex of each high-z galaxy.** Conversely, for our second local comparison sample we require the SFRs of the high-z galaxies and local counterparts to be consistent within 0.2 dex but impose no constraints on $M_∗$...*

> *To ensure that the statistical properties of the matched, local and high-z samples are equivalent, **we select the same number of local counterparts for each high-z galaxy**. Although there were more than 50 local galaxies with equivalent $M_∗$ for each high-z galaxy, a greater sample size did not result in a change in the electron-density distribution. **We therefore limit the size of our $M_∗$-matched sample, by randomly selecting 50 local galaxies for each high-z galaxy**. In contrast, the number of local counterparts in both the SFR-matched and the $M_∗$-and-SFR-matched samples is limited by the rarity of high SFR galaxies in the local SDSS sample. We only find seven local counterparts for our highest SFR high-z galaxy and thus select seven local galaxies at random for the remainder of our sample. Because we impose further constraints to select the $M_∗$-and-SFR- matched sample we are limited to five local counterparts for each high-z galaxy.*

To summarise, we should
* Determine how to quantify our independent variable (i.e. how to classify galaxies into having >1 component)
* Determine which properties on which to match our sample - e.g. stellar mass, SFR, morphology, etc. 
* After this, look at the sample sizes: how many 1-comp galaxies should we match to each 2-comp galaxy?

In [2]:
%matplotlib widget

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from astropy.visualization import hist

from spaxelsleuth.loaddata.sami import load_sami_df
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn

import matplotlib.pyplot as plt 
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [4]:
ncomponents = "recom"
bin_type = "default"
eline_SNR_min = 5

In [5]:
# Load the metadata
df_metadata = pd.read_hdf(os.path.join(os.environ["SAMI_DIR"], "sami_dr3_metadata.hd5"))
df_metadata_extended = pd.read_hdf(os.path.join(os.environ["SAMI_DIR"], "sami_dr3_metadata_extended.hd5"))

# Mask out the cell with the extremely high R_e (which is almost certainly incorrect!)
cond_bad_Re = df_metadata_extended["R_e (kpc)"] > 500
df_metadata_extended.loc[cond_bad_Re, "R_e (kpc)"] = np.nan
assert not any(df_metadata_extended.index.duplicated())

# Add data from the EmissionLine1compDR3 table
df_ap1comp = pd.read_hdf(os.path.join(os.environ["SAMI_DIR"], f"sami_apertures_extcorr_minSNR=5.hd5"))
# data_path = os.path.join(spaxelsleuth.loaddata.__file__.split("loaddata")[0], "data")
# df_ap1comp = pd.read_csv(os.path.join(data_path, "sami_EmissionLine1compDR3.csv"))
# df_ap1comp = df_ap1comp.set_index("catid").drop(["Unnamed: 0"], axis=1)
# df_ap1comp = df_ap1comp[~df_ap1comp.index.duplicated(keep='first')]  # Remove duplicate indices
assert not any(df_ap1comp.index.duplicated())

# Merge onto df_metadata_extended
df_metadata_extended = df_metadata_extended.merge(df_ap1comp[[c for c in df_ap1comp.columns if c not in df_metadata_extended.columns]], how="inner", left_index=True, right_index=True)

# Compute the mean SFR surface density within a 3kpc aperture 
df_metadata_extended["sigma_SFR (3kpc round)"] = df_metadata_extended["SFR (3kpc round)"] / (np.pi * 3**2)
df_metadata_extended["sigma_SFR (3kpc round) (error)"] = df_metadata_extended["SFR error (3kpc round)"] / (np.pi * 3**2)
df_metadata_extended["log sigma_SFR (3kpc round)"] = np.log10(df_metadata_extended["sigma_SFR (3kpc round)"])  # not bothering with computing log errors

# Specific SFR in 3kpc
df_metadata_extended["log sSFR (3kpc round)"] = np.log10(df_metadata_extended["SFR (3kpc round)"]) - df_metadata_extended["mstar"]

# Compute the mean SFR surface density within a 3kpc aperture 
df_metadata_extended["sigma_SFR (R_e)"] = df_metadata_extended["SFR (R_e)"] / (np.pi * df_metadata_extended["R_e (kpc)"]**2)
df_metadata_extended["sigma_SFR (R_e) (error)"] = df_metadata_extended["SFR error (R_e)"] / (np.pi * df_metadata_extended["R_e (kpc)"]**2)
df_metadata_extended["log sigma_SFR (R_e)"] = np.log10(df_metadata_extended["sigma_SFR (R_e)"])  # not bothering with computing log errors

# Specific SFR in 3kpc
df_metadata_extended["log sSFR (R_e)"] = np.log10(df_metadata_extended["SFR (R_e)"]) - df_metadata_extended["mstar"]


assert not any(df_ap1comp.index.duplicated())


In [6]:
df = load_sami_df(ncomponents=ncomponents, bin_type=bin_type, eline_SNR_min=eline_SNR_min, correct_extinction=True)

In load_sami_df(): Loading DataFrame...
In load_sami_df(): Finished!


### STEP 1: quantifying the variable of interest
--- 
Classify "2-component galaxies" as being galaxies in which >30% of spaxels have 2 components, and >90% of spaxels are star-forming. Maybe also remove Seyfert galaxies. 

Note that we should only use the spaxels in which the final number of components matches the original number of components.

Then, we should do a couple of checks before proceeding:
* how many galaxies are there that meet these criteria?
* plot the sSFR of these galaxies vs. stellar mass in comparison with the whole sample 

In [7]:
# Limit sample to having high-quality multi-component fits
df_hq = df[df["Number of components"] == df["Number of components (original)"]]
print(f"{df_hq.shape[0] / df.shape[0] * 100}% of spaxels have high-quality components only")

61.04448405265259% of spaxels have high-quality components only


In [9]:
# Select 2-component galaxies
gals = df_hq.catid.unique()
gals_2comp = []
gals_1comp = []
for gal in tqdm(gals):
    df_gal = df_hq[df_hq.catid == gal]
    
    # Criterion: >20% of spaxels must have >1 component 
    cond_anycomp = df_gal["Number of components"] >= 1
    n_anycomp = df_gal[cond_anycomp].shape[0]
    cond_2comp = df_gal["Number of components"] >= 2
    n_2comp = df_gal[cond_2comp].shape[0]
    
    # Criterion: of those spaxels that can be classified, >80% must be SF 
    cond_SF = df_gal["BPT (total)"] == "SF"
    cond_Seyfert = df_gal["BPT (total)"] == "Seyfert"
    cond_LINER = df_gal["BPT (total)"] == "LINER"
    cond_any = df_gal["BPT (total)"] != "Not classified"
    n_SF = df_gal[cond_SF].shape[0]
    n_Seyfert = df_gal[cond_Seyfert].shape[0]
    n_LINER = df_gal[cond_LINER].shape[0]
    n_anycat = df_gal[cond_any].shape[0]
    
    if n_anycomp > 0 and n_anycat > 50 and n_Seyfert == 0 and n_LINER == 0:
        if (n_SF / n_anycat) > 0.8:
            if ((n_2comp / n_anycomp) > 0.2):
                gals_2comp.append(gal)
            elif ((n_2comp / n_anycomp) < 0.05):
                gals_1comp.append(gal)

print(f"{len(gals_2comp)}/{len(gals)} meet our 2-component criteria")
print(f"{len(gals_1comp)}/{len(gals)} meet our 1-component criteria")

100%|██████████| 2997/2997 [00:36<00:00, 82.16it/s]

102/2997 meet our 2-component criteria
590/2997 meet our 1-component criteria


In [18]:
# Look at the 2-component galaxies 
N = 20
fig, axs = plt.subplots(nrows=len(gals_2comp[:N]), ncols=4, figsize=(18, 4 * len(gals_2comp[:N])))
for gg, gal in enumerate(gals_2comp[:N]):
    df_gal = df_hq[df_hq.catid == gal]
    plot_sdss_image(df_gal, ax=axs[gg][0])
    plot2dmap(df_gal, "Number of components", bin_type="default", survey="sami", ax=axs[gg][1], show_title=False)
    plot2dmap(df_gal, "BPT (numeric) (total)", bin_type="default", survey="sami", ax=axs[gg][2], show_title=False)
    plot2dmap(df_gal, "delta v_gas (2/1)", bin_type="default", survey="sami", ax=axs[gg][3], show_title=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
# Look at the 1-component galaxies 
N = 3
fig, axs = plt.subplots(nrows=len(gals_1comp[:N]), ncols=3, figsize=(12, 4 * len(gals_1comp[:N])))
for gg, gal in enumerate(gals_1comp[:N]):
    df_gal = df_hq[df_hq.catid == gal]
    plot_sdss_image(df_gal, ax=axs[gg][0])
    plot2dmap(df_gal, "Number of components", bin_type="default", survey="sami", ax=axs[gg][1], show_title=False)
    plot2dmap(df_gal, "BPT (numeric) (total)", bin_type="default", survey="sami", ax=axs[gg][2], show_title=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### STEP 2: select a matched sample 
---
For each galaxy, count how many other galaxies there are in SAMI with 
* $M_*$ within 0.2 dex,
* Total $\rm SFR$ within 0.2 dex, 
* $M_*$ AND total $\rm SFR$ within 0.3 dex.

Other things we might also want to match on:
* number of spaxels with high enough S/N to measure BPT categories 
* angular size, i.e. redshift 
* inclination
* morphology 
* fraction of galaxy light in the SAMI aperture: try SFR surface density rather than total SFR? Compute total SFR within 1R_e

In [10]:
plt.close("all")

In [11]:
# Plot other variables to see what could be driving the number of components present
cols = ["mstar", "SFR (R_e)", "log sigma_SFR (R_e)", "log sSFR (R_e)",
        "R_e (kpc)", "log(M/R_e)", "Morphology (numeric)", "g_i", "m_r", "mu_within_1re",
        "A_V (R_e)", "v_gas (R_e)", "sigma_gas (R_e)",
        "log N2 (R_e)", "log S2 (R_e)", "log O1 (R_e)", "log O3 (R_e)", "S2 ratio (R_e)", 
        "N2S2 (R_e)", "N2O2 (R_e)", "Dopita+2016 (R_e)",
        "Inclination i (degrees)", "z_spec", "kpc per arcsec", "ellip", "pa",
        "Median SNR (B, 1R_e)", "Median SNR (B, 1.5R_e)", "Median SNR (B, 2R_e)", "Median SNR (B, full field)",
        "Median SNR (R, 1R_e)", "Median SNR (R, 1.5R_e)", "Median SNR (R, 2R_e)", "Median SNR (R, full field)"]

In [12]:
len(cols)

34

In [14]:
#///////////////////////////////////////////////////////////////////////////
# PARAMETERS
fig_path = os.path.join(os.environ["SAMI_DIR"], "matched_samples")
N = 1  # How many galaxies to match to each

#///////////////////////////////////////////////////////////////////////////
# OBTAIN MATCHED SAMPLE
# for match_condition in ["mstar", "sfr", "mstar+sfr", "sfrdens", "mstar+sfrdens", "ssfr", "mstar+ssfr", "inc", "re", "snr", "kpc_per_as", "mstar+sfrdens+kpc_per_as", "morph", "mu_within_1re"]:
for match_condition in ["N2O2"]:

    gals_1comp_unmatched = gals_1comp.copy()  # List of 1-component galaxies that haven't been matched yet
    gals_2comp_unmatched = []  # 2-component galaxies that have < N counterparts (and are excluded from analysis)
    gals_1comp_matched = []    # Final list of 1-component galaxies that are counterparts to the 2-component galaxies 
    gals_2comp_matched = []    # Final list of 2-component galaxies that have N counterparts in our sample

    # Split the metadata DataFrame into that containing the 2-component galaxies, and all the others
    df_metadata_2comp = df_metadata_extended.loc[gals_2comp]
    df_metadata_unmatched = df_metadata_extended.loc[gals_1comp_unmatched]
    gals_matched_dict = {}

    # For each 2-comp galaxy,
    for gal in gals_2comp:
        
        # Match conditions
        # Match in mstar
        if match_condition == "mstar":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.2

        # Match in SFR 
        elif match_condition == "sfr":
            sfr = df_metadata_2comp.loc[gal, "SFR (R_e)"]
            cond_matched = np.abs(np.log10(df_metadata_unmatched["SFR (R_e)"]) - np.log10(sfr)) < 0.2

        # Match in mstar & SFR
        elif match_condition == "mstar+sfr":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            sfr = df_metadata_2comp.loc[gal, "SFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3
            cond_matched &= np.abs(np.log10(df_metadata_unmatched["SFR (R_e)"]) - np.log10(sfr)) < 0.3

        # Match in SFR surface density 
        elif match_condition == "sfrdens":
            sfrdens = df_metadata_2comp.loc[gal, "log sigma_SFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["log sigma_SFR (R_e)"] - sfrdens) < 0.2

        # Match in mstar & SFR surface density
        elif match_condition == "mstar+sfrdens":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            sfrdens = df_metadata_2comp.loc[gal, "log sigma_SFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3        
            cond_matched &= np.abs(df_metadata_unmatched["log sigma_SFR (R_e)"] - sfrdens) < 0.3

        # Match in sSFR 
        elif match_condition == "ssfr":
            ssfr = df_metadata_2comp.loc[gal, "log sSFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["log sSFR (R_e)"] - ssfr) < 0.2

        # Match in mstar & sSFR
        elif match_condition == "mstar+ssfr":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            ssfr = df_metadata_2comp.loc[gal, "log sSFR (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3         
            cond_matched &= np.abs(df_metadata_unmatched["log sSFR (R_e)"] - ssfr) < 0.3

        # Match in inclination
        elif match_condition == "inc":
            i = df_metadata_2comp.loc[gal, "Inclination i (degrees)"]
            cond_matched = np.abs(df_metadata_unmatched["Inclination i (degrees)"] - i) < 10

        # Match in R_e 
        elif match_condition == "re":
            re = df_metadata_2comp.loc[gal, "R_e (kpc)"]
            cond_matched = np.abs(df_metadata_unmatched["R_e (kpc)"] - re) < 1.0
        
        # Match in mstar & Re
        elif match_condition == "mstar+re":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            re = df_metadata_2comp.loc[gal, "R_e (kpc)"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3         
            cond_matched &= np.abs(df_metadata_unmatched["R_e (kpc)"] - re) < 1.0

        # Match in S/N
        elif match_condition == "snr":
            snr = df_metadata_2comp.loc[gal, "Median SNR (B, 1R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["Median SNR (B, 1R_e)"] - snr) < 2

        # Match in kpc per arcsec
        elif match_condition == "kpc_per_as":
            kpc_per_as = df_metadata_2comp.loc[gal, "kpc per arcsec"]
            cond_matched = np.abs(df_metadata_unmatched["kpc per arcsec"] - kpc_per_as) < 0.3

        # Match in mstar & SFR
        elif match_condition == "mstar+sfrdens+kpc_per_as":
            mstar = df_metadata_2comp.loc[gal, "mstar"]
            sfrdens = df_metadata_2comp.loc[gal, "log sigma_SFR (R_e)"]
            kpc_per_as = df_metadata_2comp.loc[gal, "kpc per arcsec"]
            cond_matched = np.abs(df_metadata_unmatched["mstar"] - mstar) < 0.3
            cond_matched &= np.abs(df_metadata_unmatched["log sigma_SFR (R_e)"] - sfrdens) < 0.3
            cond_matched &= np.abs(df_metadata_unmatched["kpc per arcsec"] - kpc_per_as) < 0.3

        # Match in surface brightness
        elif match_condition == "mu_1re":
            mu_1re = df_metadata_2comp.loc[gal, "mu_1re"]
            cond_matched = np.abs(df_metadata_unmatched["mu_1re"] - mu_1re) < 0.2
        
        # Match in surface brightness
        elif match_condition == "mu_within_1re":
            mu_1re = df_metadata_2comp.loc[gal, "mu_1re"]
            cond_matched = np.abs(df_metadata_unmatched["mu_1re"] - mu_1re) < 0.2

        # Match in surface brightness
        elif match_condition == "morph":
            morph = df_metadata_2comp.loc[gal, "Morphology (numeric)"]
            cond_matched = np.abs(df_metadata_unmatched["Morphology (numeric)"] - morph) == 0
            
        # Match in N2O2
        elif match_condition == "N2O2":
            N2O2 = df_metadata_2comp.loc[gal, "N2O2 (R_e)"]
            cond_matched = np.abs(df_metadata_unmatched["N2O2 (R_e)"] - N2O2) < 0.2

        # Get ALL of the galaxies satifying these criteria.
        gals_matched = list(df_metadata_unmatched[cond_matched].index)
        if len(gals_matched) >= N:
            # Select a random subset of these, so that individual galaxies don't "hog" them all
            gals_matched_subset = list(np.random.choice(gals_matched, size=min(N, len(gals_matched)), replace=False))

            # Add the 1-comp galaxies to the list of galaxies that have been matched 
            gals_1comp_matched += gals_matched_subset
            gals_matched_dict[str(gal)] = gals_matched_subset

            # Add the 2-comp galaxy to the list of galaxies that have been matched 
            gals_2comp_matched.append(gal)

            # Remove the newley matched galaxies from the unmatched 1-comp galaxy list 
            df_metadata_unmatched = df_metadata_unmatched.drop(gals_matched_subset, axis=0)
            for gal in gals_matched_subset:
                gals_1comp_unmatched.remove(gal)
        else:
            # Drop this galaxy, because it doesn't have enough counterparts 
            gals_2comp_unmatched.append(gal)
            print(f"{gal} only has {len(gals_matched)} 1-component counterparts")

    # Check that this list is unique so that we aren't accidentally including galaxies twice
    assert len(np.unique(gals_1comp_matched)) == len(gals_1comp_matched)

    #///////////////////////////////////////////////////////////////////////////
    # PLOT 
    fig, axs = plt.subplots(nrows=5, ncols=7, figsize=(7 * 4, 5 * 4))    
    for cc, col in enumerate(cols):
        # Plot 
        r = (vmin_fn(col), vmax_fn(col)) if vmin_fn(col) is not None else (df_metadata_extended.loc[gals_2comp_matched + gals_1comp_matched, col].min(), df_metadata_extended.loc[gals_2comp_matched + gals_1comp_matched, col].max())
        hist(df_metadata_extended.loc[gals_2comp_matched, col], histtype="step", ax=axs.flat[cc], range=r, bins=20, label=f"2-component sample (N = {len(gals_2comp_matched)})", normed=False, color="red")
        hist(df_metadata_extended.loc[gals_1comp_matched, col], histtype="stepfilled", alpha=0.3, ax=axs.flat[cc], range=r, bins=20, label=f"Matched sample (N = {len(gals_1comp_matched)})", normed=False, color="orange")

        # Compute mean & std. dev.
        y_2comp = 0.1 * axs.flat[cc].get_ylim()[1]
        y_matched = 0.15 * axs.flat[cc].get_ylim()[1]
        y_whole = 0.2 * axs.flat[cc].get_ylim()[1]
        axs.flat[cc].errorbar(x=df_metadata_extended.loc[gals_2comp_matched, col].mean(), 
                         xerr=[[df_metadata_extended.loc[gals_2comp_matched, col].mean() - np.quantile(df_metadata_extended.loc[gals_2comp_matched, col].dropna().values, 0.16)], 
                               [np.quantile(df_metadata_extended.loc[gals_2comp_matched, col].dropna().values, 0.84) - df_metadata_extended.loc[gals_2comp_matched, col].mean()]],
                         y=y_2comp, marker="o", label="Mean (2-component sample)", color="red")
        axs.flat[cc].errorbar(x=df_metadata_extended.loc[gals_1comp_matched, col].mean(), 
                         xerr=[[df_metadata_extended.loc[gals_1comp_matched, col].mean() - np.quantile(df_metadata_extended.loc[gals_1comp_matched, col].dropna().values, 0.16)], 
                               [np.quantile(df_metadata_extended.loc[gals_1comp_matched, col].dropna().values, 0.84) - df_metadata_extended.loc[gals_1comp_matched, col].mean()]],
                         y=y_matched, marker="o", label="Mean (matched sample)", color="orange")

        axs.flat[cc].set_xlabel(label_fn(col))
    axs[0][-1].legend(loc="center left", bbox_to_anchor=[1.1, 0.5])
    axs[0][0].set_title(f"Match condition: {match_condition}")

    # Save 
    fig.savefig(os.path.join(fig_path, f"matched_sample_{match_condition}.pdf"), bbox_inches="tight", format="pdf")


460374 only has 0 1-component counterparts
70114 only has 0 1-component counterparts
751043 only has 0 1-component counterparts
3630097 only has 0 1-component counterparts
3630239 only has 0 1-component counterparts
3895257 only has 0 1-component counterparts
3896912 only has 0 1-component counterparts
9403800283 only has 0 1-component counterparts
418737 only has 0 1-component counterparts
9239901557 only has 0 1-component counterparts


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/astropy/visualization/hist.py:72: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  return ax.hist(x, bins, **kwargs)


### STEP 3: make scatter plots showing the distribution of 2comp and 1comp galaxies in different planes
---

In [73]:
fig, axs = plt.subplots(nrows=1, ncols=1, squeeze=False)
df_metadata_extended["log SFR (3kpc round)"] = np.log10(df_metadata_extended["SFR (3kpc round)"])
df_metadata_extended["log SFR (total)"] = np.log10(df_metadata_extended["SFR (total)"])

col_x = "mstar"
col_y = "log SFR (3kpc round)"

axs[0][0].scatter(df_metadata_extended.loc[gals_1comp, col_x], df_metadata_extended.loc[gals_1comp, col_y], c="orange", alpha=0.4, s=20)
axs[0][0].scatter(df_metadata_extended.loc[gals_2comp, col_x], df_metadata_extended.loc[gals_2comp, col_y], c="w", edgecolors="r", s=20)
axs[0][0].set_xlabel(label_fn(col_x))
axs[0][0].set_ylabel(label_fn(col_y))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\log_{\\rm 10} \\rm (SFR \\, [M_\\odot \\, yr^{-1}])$')

In [71]:
len(gals_2comp)

104